# MORPC Insights - Employment by Industry

## Overview

The Bureau of Labor Statistics administers a program called the [Quarterly Census of Employment and Wages](https://www.bls.gov/cew/).
  
This notebook produces a tileset that includes a summary of employment by industry for the MORPC 15-county region and the counties.

## To do

 -  For the 2025 release, the input data was prepared by a manual, undocumented process.  The process should be documented (and ideally automated) for the next release.

## Setup

### Load required libraries

In [ ]:
import pandas as pd
import frictionless
import os
import sys
import datetime
import json
import textwrap
import matplotlib
from matplotlib import pyplot as plt
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

### User-specified parameters

In [ ]:
YEAR_RANGE = [2018, 2022]

### Static parameters

In [ ]:
INPUT_DIR = os.path.normpath("./input_data")

OUTPUT_DIR = os.path.normpath("./output_data")

CHART_DIRNAME = "charts"

### Define inputs

#### Create input data directory

Create input data directory if it doesn't exist.

In [ ]:
inputDir = os.path.normpath(INPUT_DIR)
if not os.path.exists(inputDir):
    os.makedirs(inputDir)

#### Summarized employment data

In [ ]:
EMP_INPUT_TABLE_FILENAME = "morpc-qcew-2018-2022-annual-MORPC15_readable.csv"
EMP_INPUT_TABLE_PATH = os.path.join(inputDir, EMP_INPUT_TABLE_FILENAME)
print("Data: {}".format(EMP_INPUT_TABLE_PATH))

#### Geography lookup table [375]

#### MORPC member list [122]

### Define outputs

#### Create output data directory

Create output data directory if it doesn't exist.

In [ ]:
outputDir = os.path.normpath(OUTPUT_DIR)
if not os.path.exists(outputDir):
    os.makedirs(outputDir)   

In [ ]:
chartDir = os.path.join(outputDir, CHART_DIRNAME)
if not os.path.exists(chartDir):
    os.makedirs(chartDir)    

## Prepare input data

### Load summarized employment data

In [ ]:
employmentRaw = pd.read_csv(EMP_INPUT_TABLE_PATH)
employmentRaw.head()

## Transform data

Filter data to exclude 'Total, all industries'

In [ ]:
filtered_data = employmentRaw.copy()

filtered_data = filtered_data[filtered_data['industry_code'] != 10]

filtered_data = filtered_data[filtered_data['own_code'] == 10]

filtered_data = filtered_data[filtered_data['agglvl_code'] == 73]

filtered_data = filtered_data.sort_values(['area_fips','year','qtr','industry_code_name'])

## Generate static charts

In [ ]:
for f in os.scandir(chartDir):
    os.remove(f)

Load a standard color set for the chart elements.  There are a lot of catagories, so combine the colors from two sets.  Ideally the colors would be selected more intentionally.

In [ ]:
import importlib
importlib.reload(morpc)

In [ ]:
colorset = json.loads(json.dumps(morpc.CONST_COLOR_CYCLES["pairing1"]))

In [ ]:
%matplotlib agg

# Iterate over each geography in data set
for county in filtered_data['county_name'].unique():
   
    county_data = filtered_data[filtered_data['county_name'] == county]
    
    # Ensure 'YEAR', 'annual_avg_emplvl_prop', and 'industry_code_name' columns are present
    county_data = county_data[['year', 'annual_avg_emplvl_prop', 'industry_code_name']]
    
    # Aggregate to handle duplicates by taking the mean
    county_data = county_data.groupby(['year', 'industry_code_name'], as_index=False).agg({'annual_avg_emplvl_prop': 'mean'})
    
    # Pivot data to have 'industry_code_name' as columns for plotting
    county_data_pivot = county_data.pivot(index='year', columns='industry_code_name', values='annual_avg_emplvl_prop')
    county_data_pivot.index.name = "Year"    

    # Normalize data for 100% stacking
    county_data_pivot_percentage = county_data_pivot.div(county_data_pivot.sum(axis=1), axis=0) * 100

    # Reverse the order of the columns so the categories appear alphabetically from top to bottom
    county_data_pivot_percentage = county_data_pivot_percentage[county_data_pivot_percentage.columns[::-1]]
    
    # Generate a title string
    title = "Employment Share by Industry - {}".format(county)
    xlabel = None
    ylabel = "Percent of employed workers"
    
    # Create and annotate the plot
    PLOTWIDTH = 8
    fig,ax = plt.subplots(figsize=(PLOTWIDTH,PLOTWIDTH/16*9))

    county_data_pivot_percentage.plot.bar(ax=ax, stacked=True, color=colorset)

    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_ylim(ymin=0, ymax=100)
    plt.xticks(rotation=30, fontsize=10)
    handles, labels = ax.get_legend_handles_labels()
    labels = [textwrap.fill(label, 24) for label in labels]
    legend = plt.legend(handles[::-1], labels[::-1], title='Industry', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    plt.tight_layout()
    
    # Format the y-axis labels as integers with comma separators
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    filename = ("Region15" if county == "15-County Region" else county.replace(" County",""))
    
    # Save the figure to disk as an SVG file
    ax.figure.savefig(os.path.join(chartDir, "shares-{}.svg".format(filename)), bbox_extra_artists=(legend,), bbox_inches='tight')
    
    plt.close(ax.figure)

    writer = pd.ExcelWriter(os.path.join(chartDir, "shares-{}.xlsx".format(filename)), engine='xlsxwriter')
    dataOptions = {
        "numberFormat": "#,##0",
        "columnWidth": 30
    }
    chartOptions = {
        "subtype":"stacked",
        "colors": colorset,
        "titles": {
            "chartTitle": title,
            "xTitle": xlabel,
            "yTitle": ylabel
        },
        "seriesOptions": [{"gap":50} for x in county_data_pivot_percentage.columns],
        "xAxisOptions": {
            "num_font": {"size":14},
            "major_gridlines": {"visible":False}
        },
        "yAxisOptions": {
            "min":0,
            "max":100,
            "num_font": {"size":14},
            "name_font": {"size": 14},
            "num_format": "#,##0",
            "major_gridlines": {"visible":False}            
        },
        "legendOptions":{
            "position":"right",
            "font":{"size":14}
        },
        "sizeOptions":{
            "x_scale":2,
            "y_scale":2
        },
        "location": "below"
    }
    morpc.data_chart_to_excel(county_data_pivot_percentage, writer, chartType="column", dataOptions=dataOptions, chartOptions=chartOptions)
    writer.close()    
    
%matplotlib inline    

## Generate Insights catalog content

The content in the Insights platform is controlled by a catalog spreadsheet. Each tile to be displayed in the platform must have a record in the catalog.  This section will create the records for the tiles that display the alternative fuel station data.  Eventually this function will be performed by a separate staging script.

First specify the column names used in the catalog.

In [ ]:
columnNames=["TileID","TilesetID","GeographyType","GeographyName","Category","Headline","Commentary","ThumbnailURL","Contributor","Vintage","UpdateInterval","ShareURL","DataProductURL","MoreInformationURL"]

Create a new dataframe for the catalog.

In [ ]:
catalog = pd.DataFrame(index=range(0,16), columns=columnNames)

Add the names of the counties and the region.

In [ ]:
catalog["GeographyName"] = filtered_data["county_name"].unique()

Add the geography types.

In [ ]:
catalog["GeographyType"] = catalog["GeographyName"].apply(lambda name:("Region" if name == "15-County Region" else "County"))

Add a temporary column to use to populate target URLs.

In [ ]:
catalog["GEOLABEL"] = catalog["GeographyName"].apply(lambda name:("Region15" if name == "15-County Region" else name.replace(" County","")))

Populate some placeholder fields.

In [ ]:
catalog["TileID"] = None
catalog["TilesetID"] = "TBD-Shares"
catalog["Category"] = None
catalog["Headline"] = "TBD"
catalog["Commentary"] = "TBD"

Generate the URL for the thumbnail images. These will be hosted in GitHub and will be indexed by GEOLABEL.

In [ ]:
catalog["ThumbnailURL"] = catalog["GEOLABEL"].apply(lambda name:"https://raw.githubusercontent.com/morpc-insights/employment-industry/refs/heads/main/output_data/charts/shares-{}.svg".format(name))

Populate some other simple metadata.  Vintage in this case refers to the year that the content was published in Insights. This is to give readers an idea of how old it is.  UpdateInterval gives them an idea of when to expect the next version. ShareURL is a placeholder for now.

In [ ]:
catalog["Contributor"] = "Mid-Ohio Regional Planning Commission"
catalog["Vintage"] = str(datetime.date.today().year)
catalog["UpdateInterval"] = "annually"
catalog["ShareURL"] = None

Generate the data product URL.  This points to an ArcGIS Dashboard that accepts URL parameters.  GEOLABEL is passed as a parameter to tell the app to load the data for a particular geography.

In [ ]:
catalog["DataProductURL"] = catalog["GeographyName"].apply(lambda name:"https://www.arcgis.com/apps/dashboards/f939956abd634937b0e875b7a277aec9#region={}".format(name.replace(" ","%20")))

Capture the fact sheet URL.  This points to a single document for all tiles in the tileset.

In [ ]:
catalog["MoreInformationURL"] = "https://morpc1-my.sharepoint.com/:w:/g/personal/aporr_morpc_org/EZcbpRmm-OVAloxLS3ZdKuIBEXDlLLfcxc66yA4282nWaA?e=DL4oZ1"

Extract only the required columns.

In [ ]:
catalog = catalog.filter(items=columnNames, axis="columns")

Inspect the listing.

In [ ]:
catalog.head()

Save the catalog to an Excel spreadsheet.

In [ ]:
catalog.to_excel("catalog.xlsx", index=False)

It is necessary to manually add these records to the master catalog or update the records already therein.  See the following file in GitHub:

https://github.com/morpc/morpc-insights/blob/main/catalog/morpc_insights_catalog.xlsx
